In [1]:
from diel_models.pipeline import Pipeline
import cobra
import os
from tests import TEST_DIR
from diel_models.day_night_creator import DayNightCreator
from diel_models.storage_pool_generator import StoragePoolGenerator
from diel_models.photon_reaction_inhibitor import PhotonReactionInhibitor
from diel_models.biomass_regulator import BiomassAdjuster
from diel_models.nitrate_uptake_ratio import NitrateUptakeRatioCalibrator

aragem_model_path = os.path.join(TEST_DIR, 'data', 'aragem_photo.xml')

modelo = cobra.io.read_sbml_model(aragem_model_path)

Steps = [
    DayNightCreator(modelo),
    StoragePoolGenerator(modelo, ["S_Sucrose_c[C_c]_Day", "S_Sulfate_c[C_c]_Day", "S_Nitrate_c[C_c]_Day", "S_L_45_Histidine_c[C_c]_Day", "S_L_45_Isoleucine_c[C_c]_Day", "S_L_45_Leucine_c[C_c]_Day", "S_L_45_Lysine_c[C_c]_Day", "S_L_45_Methionine_c[C_c]_Day", "S_L_45_Phenylalanine_c[C_c]_Day", "S_L_45_Threonine_c[C_c]_Day", "S_L_45_Tryptophan_c[C_c]_Day", "S_L_45_Valine_c[C_c]_Day", "S_L_45_Arginine_c[C_c]_Day", "S_L_45_Cysteine_c[C_c]_Day", "S_L_45_Glutamine_c[C_c]_Day", "S_L_45_Glutamate_c[C_c]_Day", "S_Glycine_c[C_c]_Day", "S_L_45_Proline_c[C_c]_Day", "S_L_45_Tyrosine_c[C_c]_Day", "S_L_45_Alanine_c[C_c]_Day", "S_L_45_Asparagine_c[C_c]_Day", "S_L_45_Serine_c[C_c]_Day", "S_Orthophosphate_c[C_c]_Day", "S_Starch_p[C_p]_Day", "S_D_45_Fructose_c[C_c]_Day", "S__40_S_41__45_Malate_c[C_c]_Day", "S_Fumarate_c[C_c]_Day", "S_Citrate_c[C_c]_Day"]),
    PhotonReactionInhibitor(modelo, "Ex16_Night"),
    BiomassAdjuster(modelo, "BIO_L_Day", "BIO_L_Night", ["R03845_c_Night", "R06267_c_Night", "R06286_c_Night", "R04801_c_Night", "R07856_c_Night", "R04803_c_Night", "R06961_c_Night", "R06960_c_Night", "R07856_c_Night", "V0004_Night", "R06284_c_Night", "R05618_c_Night", "R03845_c_Night", "R03824_c_Night", "R05341_c_Night", "R06965_c_Night", "R07558_c_Night", "R04798_c_Night", "R04787_c_Night", "R07840_c_Night", "R06963_c_Night", "R06962_c_Night", "R06961_c_Night"]),
    NitrateUptakeRatioCalibrator(modelo, "Ex4_Day", "Ex4_Night")
]

pipeline = Pipeline(modelo, Steps)
pipeline.run()

ModuleNotFoundError: No module named 'numpy'

In [3]:
res_model = pipeline.model
res_model

Name,M_
Memory address,23a24aaddc0
Number of metabolites,3474
Number of reactions,3259
Number of genes,1404
Number of groups,306
Objective expression,1.0*Biomass_Total - 1.0*Biomass_Total_reverse_a0aa3
Compartments,"unknownCompartment3 Day, unknownCompartment6 Day, unknownCompartment5 Day, unknownCompartment8 Day, unknownCompartment7 Day, unknownCompartment2 Day, unknownCompartment1 Day, unknownCompartment4 Day, unknownCompartment3 Night, unknownCompartment2 Night, unknownCompartment6 Night, unknownCompartment1 Night, unknownCompartment4 Night, unknownCompartment5 Night, unknownCompartment7 Night, unknownCompartment8 Night"
